# Counting distinct values

It's extremely common for analysts to want to count unique occurences of some result set or function. The Druid database enables you to leverage advanced computer science techniques to speed up this type of calculation through approximation.

In this tutorial, work through some examples and see the effect of turning it on and off, and of making it even faster by pre-generating the objects that Druid uses to execute the query.

## Prerequisites

This tutorial works with Druid 26.0.0 or later.

#### Run using Docker

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see [Docker for Jupyter Notebook tutorials](https://druid.apache.org/docs/latest/tutorials/tutorial-jupyter-docker.html).
   
#### Run without using Docker

If you do not use the Docker Compose environment, you need the following:

* A running Apache Druid instance, with a `DRUID_HOST` local environment variable containing the servername of your Druid router
* [druidapi](https://github.com/apache/druid/blob/master/examples/quickstart/jupyter-notebooks/druidapi/README.md), a Python client for Apache Druid. Follow the instructions in the Install section of the README file.
* [matplotlib](https://matplotlib.org/), a library for creating visualizations in Python,
* [pandas](https://pandas.pydata.org/), a data analysis and manipulation tool.

### Initialize Python

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"
    
print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)

display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Load example data

Once your Druid environment is up and running, ingest the sample data for this tutorial.

Open the Druid console, and ingest the data as follows:

1. Select **Load data** from the top-level navigation.
2. Select **Batch - SQL**.
3. For the input type, select **Example data**.
4. Select **FlightCarrierOnTime (1 month)**.
5. Click **Use example**.

Go through the data loader wizard using the defaults, including the datasource name:

`On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11`

Run the following cell to describe the table, a handy way to check that the table that you will need for this notebook is present.

In [ ]:
display.table('On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11')

Finally, run the following cell to import additional Python modules that you will use.

In [ ]:
import json
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## Using COUNT(DISTINCT)

Finding the number of distinct elements in a set is very common - a `COUNT(DISTINCT)` function, like the one below, allows analysts to determine how many individual things, like a visitor, a device, or an airplane (!), appear in a set. 

```sql
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Events"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
```

In Druid, each data server computes its own intermediate results that are then merged into a final result set. In a `COUNT(DISTINCT)` query, each process its own set of results, passes these back, and then computation is done to find the result. When the dimension being looked at contains many tens-of-thousands, perhaps even millions of unique values, the transfer of intermediate results and the final computation can be both memory and CPU hungry.

During planning, Druid looks for query patterns that benefit from from approximation, like `COUNT(DISTINCT)`. You can therefore handle these types of queries in two modes – with and without [Apache Datasketch](https://datasketches.apache.org/)-based approximation.

### COUNT(DISTINCT) with approximation

With approximation, intermediate results from each data process are put into a representation called a [data sketch](https://datasketches.apache.org/). These objects are transfered and merged together as the queries complete. Data sketch size is not dependent on the underlying data – the default size of a sketch in Druid is just over 2000 bytes.

This translates into much faster query execution, especially when the intermediate results are large. The most scalable part of Druid – the data processes – do much more of the work, and do so earlier.

> Approximations improve scalability, storage, and memory use - at the cost of some error.
> 
> _[Gian Merlino](https://github.com/gianm)_

By default, each Druid process will be calculating its local results as data sketches, passing these back for the final merge operation on which size estimation is performed.

You can look into Druid's [configuration files](https://druid.apache.org/docs/26.0.0/configuration/index.html#sql) to find whether this approach has been left as the default by your system administrators (`druid.sql.planner.useApproximateCountDistinct`) and what type of data sketch will be used (`druid.sql.approxCountDistinct.function`).

Run the following cell to execute a `COUNT(DISTINCT)` query. It finds the number of unique `Tail_Number`s for each `Reporting_Airline` and stores the results in a dataframe. The results are then plotted in a histogram.

With default settings, this query will execute using approximation.

In [ ]:
sql = '''
SELECT
    "Reporting_Airline",
    COUNT(DISTINCT "Tail_Number") AS "Unique Tail Numbers"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2
'''

df1 = pd.DataFrame(sql_client.sql(sql))

df1.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### COUNT(DISTINCT) without approximation

Supply a query context parameter, `useApproximateCountDistinct`, to force Druid to not use approximation for `COUNT(DISTINCT)` queries.

Using the same SQL statement as before, the following cell crafts a request (`req`). The request is then given a context parameter to turn off approximation. The response is then stored and put into a second dataframe, from which we get a plot of unique `Tail_Number`s by `Reporting_Airline`.

In [ ]:
req = sql_client.sql_request(sql)
req.add_context("useApproximateCountDistinct", "false")
resp = sql_client.sql_query(req)

df2 = pd.DataFrame(resp.rows)
df2.plot.bar(x='Reporting_Airline', y='Unique Tail Numbers')
plt.show()

### Comparing approximate and non-approximate results

The next cell shows a comparison of the two results above: `df1` used the default approximation approach, while `df2` are the results where we turned approximation off.

In [ ]:
df3 = df1.compare(df2, keep_equal=True)
df3

The table shows:

* A row number
* The reporting airline in the approximate results (`self`) versus that in the non-approximate results (`other`)
* The calculated distinct number of `Tail Number`s

Notice that there are _value_ errors, as you might expect with approximation, and that in some instances this affects the _order_ of results.

Error in sketch-based approximation is probabilistic, rather than guaranteed. That's to say that a certain percentage of the time you can expect the measurements you take to be within a certain distance of the true value.

## Calculating set union with Theta and HyperLogLog sketches

There are two types of Apache Datasketch you can use to estimate the size of a union of one or more sets:

* [HyperLogLog](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#hll-sketch-functions)
* [Theta](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#theta-sketch-functions)

Each allows Druid to estimate the `COUNT(DISTINCT)` of the union of two or more sets. When you ran the `COUNT(DISTINCT)` query in approximate mode, Druid arrived at a single set that was the union of the intermediate sets, and returned an estimate of the set size.

In Druid SQL, you can access functions that allow you to define your own sets that you can union in order to estimate their size.

Run the next cell, which:

* Gets three sets of `Tail_Number`s using `DS_HLL` - it applies a `FILTER` to isolate flights out of three specific cities,
* Applies `HLL_SKETCH_UNION` to union the three sets, and
* Estimates the resulting set size with `HLL_SKETCH_ESTIMATE`.

It uses `TIME_FLOOR` to giving us a week-by-week `GROUP BY` of the data.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'P1W') AS "Week commencing",
  HLL_SKETCH_ESTIMATE(
     HLL_SKETCH_UNION(
       DS_HLL("Tail_Number") FILTER (WHERE "Origin"='ATL'),
       DS_HLL("Tail_Number") FILTER (WHERE "Origin"='DFW'),
       DS_HLL("Tail_Number") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "AnyCity-HLL",
  THETA_SKETCH_ESTIMATE(
     THETA_SKETCH_UNION(
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='ATL'),
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='DFW'),
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "AnyCity-Theta"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE TIMESTAMP '2005-10-31' <= __time AND __time <= TIMESTAMP '2005-11-20'
GROUP BY 1
'''

display.sql(sql)

Because of differences in how HyperLogLog and Theta sketch functions themselves work, and defaults of how intermediate sketches themselves are constructed at query-time, there are differences in the results between HyperLogLog and Theta sketches.

Read more about this in the [documentation](https://druid.apache.org/docs/26.0.0/querying/sql-aggregations.html#hll-sketch-functions) for the `DS_HLL` and `DS_THETA` functions.

## Calculating set intersection and difference with Theta sketches

With Theta sketches, you can also approximate the size of:

* The intersection of two sets (airplanes that went to both ATL _and_ SFO)
* The difference between one set and another (airplanes that went to ATL and _not_ SFO)


### Set intersection

Run the next cell to see the intersection between three Theta sketch sets, week-by-week.

As in the query above, each set is filtered to specific airports, then an intersection is performed, before finally the size of that set is estimated and passed back.

In [ ]:
sql='''
SELECT
  TIME_FLOOR("__time",'P1W') AS "Week commencing",
  THETA_SKETCH_ESTIMATE(
     THETA_SKETCH_INTERSECT(
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='ATL'),
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='DFW'),
       DS_THETA("Tail_Number") FILTER (WHERE "Origin"='SFO')
      )
    ) AS "AllThreeCities"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
WHERE TIMESTAMP '2005-10-31' <= __time AND __time <= TIMESTAMP '2005-11-20'
GROUP BY 1
'''

display.sql(sql)

This is an important application of set operations: estimating how many of some object carried out some action or other. In the example above, that is how many airplanes took off from from all three cities by week.

Another application might be to use Druid's event analytics capabilities to look for time-based intersections.

The next cell creates a dataframe from SQL that creates two sets using the `FILTER` function. The first set represents all airplanes that flew on the week commencing 31st October, and the second for the week commencing 7th November. It then intersects these to create a new sketch representing all the airplanes that flew on both days. Finally, the size of that set is estimated, with a `GROUP BY` that breaks it down by `Reporting_Airline`.

In [ ]:
sql='''
SELECT
  "Reporting_Airline",
  THETA_SKETCH_ESTIMATE(
     THETA_SKETCH_INTERSECT(
       DS_THETA("Tail_Number") FILTER (WHERE TIME_FLOOR("__time",'P1W') = TIMESTAMP '2005-10-31'),
       DS_THETA("Tail_Number") FILTER (WHERE TIME_FLOOR("__time",'P1W') = TIMESTAMP '2005-11-07')
      )
    ) AS "BothWeeks"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2 DESC
'''

df = pd.DataFrame(sql_client.sql(sql))

df.plot.bar(x='Reporting_Airline', y='BothWeeks')
plt.show()

### Set difference

Finally, we turn to using Theta sketch operations to estimate the size of the difference between one set and another.

The next cell switches the `THETA_SKETCH_INTERSECT` operation, which intersects the sets, for `THETA_SKETCH_NOT`, which does a difference operation. The plot we see therefore charts, approximately, how many airplanes flew in the week commencing 31st October that did _not_ also fly in the next weeek.

Note that this operation is not cumutative - Druid calculates the size of the difference (A to B) per airline: it is not a symetric difference operation.

In [ ]:
sql='''
SELECT
  "Reporting_Airline",
  THETA_SKETCH_ESTIMATE(
     THETA_SKETCH_NOT(
       DS_THETA("Tail_Number") FILTER (WHERE TIME_FLOOR("__time",'P1W') = TIMESTAMP '2005-10-31'),
       DS_THETA("Tail_Number") FILTER (WHERE TIME_FLOOR("__time",'P1W') = TIMESTAMP '2005-11-07')
      )
    ) AS "FirstNotSecondWeek"
FROM "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_11"
GROUP BY 1
ORDER BY 2 DESC
'''

df = pd.DataFrame(sql_client.sql(sql))

df.plot.bar(x='Reporting_Airline', y='FirstNotSecondWeek')
plt.show()

## Conclusion

* Approximation is the default execution model for `COUNT(DISTINCT)` queries
* You can turn it off with a query context parameter
* Accuracy is highly dependent on the distribution and cardinality of data across the database
* HyperLogLog and Theta sketches both allow you to approximate `COUNT(DISTINCT)` of entire sets
* Only Theta sketches allow you to carry out set operations

## Learn more

* Try estimation on your own dataset:
    * Identify a high-cardinality column in one of your own data sets
    * Test how long an approximate `DISTINCT(COUNT)` query takes to run with approximation turned on
    * Test how long the same query takes to run with approximation turned off
* Watch [Casting the spell: Apache Druid in practice](https://youtu.be/QAitmv8QRq4) by Itai Yaffe and Yakir Buskilla (Nielsen)
* Watch [Employ Approximation](https://youtu.be/il84eH0kUyc) by Peter Marshall (Imply)
* Watch [Advertiser audience forecasting with Apache Druid](https://youtu.be/7PRWDMRSAOw) by Qasim Zeeshan and Sundeep Yedida (Reddit)
* Watch [Funnel Analysis in Mobile Gaming - leveraging approximation algorithms for low latency analytics](https://youtu.be/il84eH0kUyc) by Ramón Lastres Guerrero (Game Analytics)